In [1]:
from sklearn.externals import joblib
import sys
default_stdout = sys.stdout
default_stderr = sys.stderr
reload(sys)
sys.stdout = default_stdout
sys.stderr = default_stderr
sys.setdefaultencoding('utf-8')
from konlpy.tag import Twitter
twitter = Twitter()

In [2]:
import re

def filtering(name, excepts = []):
    for exce in excepts:
        name = re.sub(exce, " ", name).lstrip();
    return name
    
def ngram(n, name):
    modified_name = ""
    i = 0
    while i < len(name)-1:
        tmp = " "
        loop = i
        count = 0
        while(count < n):
            if  loop >= len(name) or name[loop] == ' ':
                tmp = " "
                break;
            else:
                tmp += name[loop]
                loop += 1
            count += 1
        modified_name += tmp
        i += 1
    return modified_name

excepts = ["\[", "\]", "\(", "\)"]

'\nexcepts = [u"\xed\x95\xb4\xec\x99\xb8", u"\xec\xa6\x89\xec\x8b\x9c", \n           u"\xed\x95\xa0\xec\x9d\xb8",\n           u"(\xea\xb5\xad\xeb\xaf\xbc|\xed\x95\x98\xeb\x82\x98|\xec\x9a\xb0\xeb\xa6\xac|\xeb\x86\x8d\xed\x98\x91|\xec\x8b\xa0\xed\x95\x9c|KB)\xec\xb9\xb4\xeb\x93\x9c", "\\[", "\\]", "\\(", "\\)", \n           u"[\xe2\x97\x80\xe2\x96\xb6\xe2\x96\xb6\xe2\x99\xa5\xe2\x97\x8f\xef\xbc\x8b\xe2\x94\x8f\xe2\x96\xa9\xe2\x94\x93\xe2\x98\x86\xe2\x96\x92\xe2\x98\x9e\xe2\x95\x8b\xe6\x9c\x89\xe2\x98\x9c\xe5\x8f\xa3~\xe7\x84\xa1+.,!@#$%^&*-=\\_+|\xe2\x94\x81\xe2\x98\x85\xe2\x96\xa0\xe2\x99\xa3\xe2\x94\x95\xce\x9b\xce\x9f\xce\x9b\xe2\x94\x99]"];\n'

In [3]:
def url_parse(url):
    u = re.split('/', url)
    #print re.split('\.', u[4])[0]
    try:
        t = re.split('\.', u[4])[0]
    except Exception:
        t = ""
    return t

url_parse('img=http://shopping.phinf.naver.net/main_8275919/8275919908.jpg')

'8275919908'

In [4]:
import json

def many_cate(num, cate):
    i = 0
    tmp = ""
    while(i < num):
        tmp += " " + cate
        i += 1
    return tmp

def image_parse(name, image_name):
    try:
        file_path = './server/' + str(image_name) + '.json'
        f = open(file_path, 'r')
        json_data = ""
        for each in f.readlines():
            json_data += each    
        j = json.loads(json_data)
        for each in j['result'][1]:
            if float(each[1]) < 0.5:
                name += many_cate(1, each[0])
            elif float(each[1]) < 1.0:
                name += many_cate(1, each[0])        
            elif float(each[1]) < 1.5:
                name += many_cate(1, each[0])
            elif float(each[1]) < 2.0:
                name += many_cate(1, each[0])
            elif float(each[1]) >= 2.0:
                name += many_cate(1, each[0])
            break
        for each in j['result'][2]:
            if float(each[1]) < 0.5:
                name += many_cate(1, each[0])
            elif float(each[1]) < 1.0:
                name += many_cate(1, each[0])        
            elif float(each[1]) < 1.5:
                name += many_cate(1, each[0])
            elif float(each[1]) < 2.0:
                name += many_cate(1, each[0])
            elif float(each[1]) >= 2.0:
                name += many_cate(1, each[0])
            break
    except Exception:
        pass
    #f.close()
    return name


In [5]:
print image_parse('[\ud574\uc678] New Fashion Womens Slim Chiffon Tops Pocket Sleeveless Shirt Casual Blouse Vest S-XXL FE3112', url_parse('http://shopping.phinf.naver.net/main_8313513/8313513096.jpg'))

[\ud574\uc678] New Fashion Womens Slim Chiffon Tops Pocket Sleeveless Shirt Casual Blouse Vest S-XXL FE3112


In [6]:

clf = joblib.load('classify.model')
cate_dict = joblib.load('cate_dict.dat')
vectorizer = joblib.load('vectorizer.dat')


In [7]:
joblib.dump(clf,'n_classify.model')

['n_classify.model',
 'n_classify.model_01.npy',
 'n_classify.model_02.npy',
 'n_classify.model_03.npy']

In [8]:
joblib.dump(cate_dict,'n_cate_dict.dat')
joblib.dump(vectorizer,'n_vectorizer.dat')

['n_vectorizer.dat']

In [9]:
cate_id_name_dict = dict(map(lambda (k,v):(v,k),cate_dict.items()))

In [ ]:
pred = clf.predict(vectorizer.transform(['[신한카드5%할인][서우한복] 아동한복 여자아동 금나래 (분홍)']))[0]
print cate_id_name_dict[pred]

패션의류;아동의류;한복


In [ ]:
from bottle import route, run, template,request,get, post


import  time
from threading import  Condition
_CONDITION = Condition()
@route('/classify')
def classify():
    print "classify called"
    img = request.GET.get('img','')
    name = request.GET.get('name', '')
    tmp = ""
    for each in twitter.morphs(filtering(name)):
        tmp += " " + each
    name = (filtering(tmp, excepts) + ' ' + name)
    name = image_parse(name, url_parse(img))
    pred = clf.predict(vectorizer.transform([name]))[0]
    return {'cate':cate_id_name_dict[pred]}


run(host='0.0.0.0', port=8887)


Bottle v0.12.9 server starting up (using WSGIRefServer())...
Listening on http://0.0.0.0:8887/
Hit Ctrl-C to quit.



 * 추후 여기 docker 에서 뭔가 python package 설치할게 있으면 
 * /opt/conda/bin/pip2 install bottle 이런식으로 설치 가능